## Example of generating necessary FCC crystal data from Onsager code to run the symmetric neural networks for 8x8x8 primitive supercells.

In [1]:
from onsager import crystal, supercell, cluster
import numpy as np
import h5py

In [2]:
N_units = 8 # No. of unit cells along each axis in the supercell
MaxOrder = 2
MaxOrderTrans = 3

a0 = 1.0
cut = 1.01*a0/np.sqrt(2)  # Nearest neighbor cutoff

crys = crystal.Crystal.FCC(a0, chemistry="A")
lattice = crys.lattice
jnetFCC = crys.jumpnetwork(0, cut)

superlatt = N_units * np.eye(3, dtype=int)
superFCC = supercell.ClusterSupercell(crys, superlatt)
# Since the "spectator" tuple in the argument is left empty, all
# sites of chemistry 0 will be mobile sites.

vacsite = cluster.ClusterSite((0, 0), np.zeros(3, dtype=int))
vacsiteInd = superFCC.index(np.zeros(3, dtype=int), (0, 0))[0]
assert vacsiteInd == 0

In [3]:
# Next, for every site, make indices
# Since we have a mono-atomic lattice, we only need to note the
# lattice vectors
RtoSiteInd = np.zeros((N_units, N_units, N_units), dtype=int)
SiteIndtoR = np.zeros((N_units*N_units*N_units, 3), dtype=int)

for i in range(N_units):
    for j in range(N_units):
        for k in range(N_units):
            Rsite=np.array([i,j,k])
            siteInd = superFCC.index(Rsite, (0,0))[0]
            RtoSiteInd[i,j,k] = siteInd
            SiteIndtoR[siteInd] = Rsite

In [4]:
# Convert the jump displacements into lattice translation
dxR = np.zeros((len(jnetFCC[0]), 3), dtype=int)
dxList = np.zeros((len(jnetFCC[0]), 3))
jList = np.zeros(len(jnetFCC[0]), dtype=int)
for jInd, ((i, j), dx) in enumerate(jnetFCC[0]):
    Rnn = np.dot(np.linalg.inv(crys.lattice), dx).round(decimals=3).astype(int)
    dxR[jInd] = Rnn
    dxList[jInd] = dx
    Rnn %= N_units
    siteIndNgb = RtoSiteInd[Rnn[0], Rnn[1], Rnn[2]]
    jList[jInd] = siteIndNgb

In [5]:
N_ngb = len(dxR)
N_ngb

12

In [6]:
# Next, for each site, store the nearest neighbors
Nsites = len(superFCC.mobilepos)
NNList = np.zeros((N_ngb + 1, Nsites), dtype=int)
for siteInd in range(Nsites):
    NNList[0, siteInd] = siteInd
    Rsite = SiteIndtoR[siteInd]
    for ngbInd, Rnn in enumerate(dxR):
        RsiteNN = (Rsite + Rnn)%N_units
        siteIndNN = RtoSiteInd[RsiteNN[0], RsiteNN[1], RsiteNN[2]]
        NNList[ngbInd + 1, siteInd] = siteIndNN

In [7]:
## Since the onsager code does not store group operations in any particular order,
## We'll give them indices and store them for further use.
GIndtoGDict = {}
count = 0
for g in crys.G:
    # Check if the group op is identity - give it the zero index - not necessary, just a choice
    if np.allclose(g.cartrot, np.eye(3)):
        GIndtoGDict[0] = g
    else:
        count += 1
        GIndtoGDict[count] = g

In [8]:
len(GIndtoGDict)

48

In [9]:
# Next, store the cartesian rotation matrices of the group operations.
GroupOpLatticeCartRotMatrices = np.zeros((len(GIndtoGDict), 3, 3))
for key in GIndtoGDict.keys():
    GroupOpLatticeCartRotMatrices[key, :, :] = GIndtoGDict[key].cartrot

In [10]:
# For each group operation, store the permuation of the nearest neighbors
# First assign indices to nearest neighbor vectors
nntoIdx = {}
for ngbInd, Rnn in enumerate(dxR):
    RnnTup = tuple(Rnn)
    nntoIdx[RnnTup] = ngbInd+1

In [11]:
# Now for each group operation, store, the permutation of the nearest neighbors due to its inverse
GpermNNIdx = np.zeros((len(GIndtoGDict), N_ngb + 1), dtype=np.uint8)
for gInd, g in GIndtoGDict.items():
    for nnTup, nnIdx in nntoIdx.items():
        nnVec = np.array(nnTup)
        nnRot, _ = crys.g_pos(g.inv(), nnVec, (0, 0))
        nnIdxNew = nntoIdx[tuple(nnRot)]
        GpermNNIdx[gInd, nnIdx] = nnIdxNew

In [12]:
# store how each jump changes occupancies, if we keep the vacancy at the origin
Nsites = SiteIndtoR.shape[0]
Njumps = dxR.shape[0]
jumpSiteIndex = np.zeros((Njumps, Nsites), dtype=int)
for jumpInd in range(Njumps):
    Rjump = dxR[jumpInd]%N_units
    RjumpNeg = -dxR[jumpInd]%N_units
    siteExchange = RtoSiteInd[Rjump[0], Rjump[1], Rjump[2]]
    siteExchangeNew = RtoSiteInd[RjumpNeg[0], RjumpNeg[1], RjumpNeg[2]]
    jumpSiteIndex[jumpInd, siteExchangeNew] = siteExchange  # the exchanged site
    for siteInd in range(1, Nsites): # vacancy doesn't move so don't include it
        if siteInd == siteExchange:
            continue
        Rsite = SiteIndtoR[siteInd]
        RsiteNew = (Rsite - dxR[jumpInd])%N_units
        siteIndNew = RtoSiteInd[RsiteNew[0], RsiteNew[1], RsiteNew[2]]
        jumpSiteIndex[jumpInd, siteIndNew] = siteInd

In [13]:
with h5py.File("CrystData.h5", "w") as fl:
    fl.create_dataset("Lattice_basis_vectors", data=lattice)
    fl.create_dataset("SuperLatt", data=superlatt)
    fl.create_dataset("SiteIndToR", data=SiteIndtoR)
    fl.create_dataset("RToSiteInd", data=RtoSiteInd)
    fl.create_dataset("GroupOpLatticeCartRotMatrices", data=GroupOpLatticeCartRotMatrices)
    fl.create_dataset("dxList_1nn", data=dxList)
    fl.create_dataset("NNsiteList_sitewise", data=NNList)
    fl.create_dataset("JumpSiteIndexPermutation", data=jumpSiteIndex)
    fl.create_dataset("GroupNNPermutation", data=GpermNNIdx)